In [16]:

from fischer.factoriobalancers.beltgraph import BeltGraph, Evaluation
import json

from sympy import isprime

class GraphDesigner:
    def __init__(self):
        self.reset()
    def reset(self):
        self.graph = BeltGraph()
        self.graph.add_edge(0, 1)
        self.graph.add_edge(1, 2)
        self.graph.add_edge(1, 3)
        self.graph.set_input(0)
        self.graph.set_output(2)
        self.graph.set_output(3)
        self.is_in_vertex_mode = True
        self.action_stack = ''
    def copy(self) -> 'GraphDesigner':
        des = GraphDesigner()
        des.graph = self.graph.copy_graph()
        des.is_in_vertex_mode = self.is_in_vertex_mode
        des.action_stack = self.action_stack
        return des
    def next_vertex(self):
        deepest = self.deepest_vertex()
        v = self.graph.new_vertex()
        self.graph.add_vertex(v)
        self.graph.disconnect_output(max(self.graph.out_vertices(deepest)))
        self.graph.add_edge(deepest, v)
        out1 = self.graph.new_vertex()
        self.graph.add_vertex(out1)
        out2 = self.graph.new_vertex()
        self.graph.add_vertex(out2)
        self.graph.add_edge(v, out1)
        self.graph.add_edge(v, out2)
        self.graph.set_output(out1)
        self.graph.set_output(out2)
        self.graph = self.graph.rearrange_vertices_by_depth()
    def preceding_vertex(self, amount):
        depths = self.graph.get_vertex_depths()
        preceding_vertex = self.deepest_vertex()
        for _ in range(amount):
            preceding_vertices = self.graph.in_vertices(preceding_vertex)
            if len(preceding_vertices) == 1:
                preceding_vertex = preceding_vertices[0]
            else:
                if depths[preceding_vertices[0]] < depths[preceding_vertices[1]]:
                    preceding_vertex = preceding_vertices[0]
                else:
                    preceding_vertex = preceding_vertices[1]
        return preceding_vertex
    def deepest_vertex(self):
        depths = self.graph.get_vertex_depths()
        return max(((k, v) for k, v in depths.items() if not self.graph.is_output(k)), key=lambda t: t[1])[0]
    def connect_backward(self, amount_from, amount_to):
        if amount_from == amount_to:
            raise Exception(f'Cannot connect a vertex to itself.')
        pv_from = self.preceding_vertex(amount_from)
        pv_to = self.preceding_vertex(amount_to)
        self.graph.disconnect_output(self.graph.out_vertices(pv_from)[0])
        self.graph.add_edge(pv_from, pv_to)
        self.is_in_vertex_mode = False
    def action(self, a: str, reset: bool = True):
        if reset:
            # print(f'Executing "{a}" from reset state.')
            self.reset()
        else:
            pass
            # print(f'Executing "{a}" from current state.')
        if self.action_stack != '':
            self.action_stack += ' -> '
        self.action_stack += a
        split = a.split(' -> ')
        for _a in split:
            if _a == 'nv':
                self.next_vertex()
            elif _a[:2] == 'cb':
                f, t = map(int, _a.split(' ')[1:])
                self.connect_backward(f, t)
    def open_outputs(self):
        for v in self.graph.outputs:
            if all(self.graph.in_vertices(self.graph.in_vertices(v)[0])): # basically just find outputs that aren't the 1/2 flow output from the very first non-input vertex
                yield self.graph.in_vertices(v)[0]
    def open_inputs(self):
        for v in self.graph.vertices():
            if not self.graph.is_input(v) and not self.graph.is_output(v) and self.graph.in_degree(v) < 2:
                yield v
    def get_cb_amount(self, v: int) -> int:
        depths = self.graph.get_vertex_depths()
        m = max(d for k, d in depths.items() if not self.graph.is_output(k))
        return m - depths.get(v, m)
    def possible_backward_connections(self):
        for u in map(self.get_cb_amount, set(self.open_outputs())):
            for v in map(self.get_cb_amount, set(self.open_inputs())):
                if u < v:
                    yield u, v
    def bfs(self):
        self.action('nv', reset=True)
        visited = [self]
        queue: list[GraphDesigner] = [self]
        while len(queue) > 0:
            v = queue[0]
            del queue[0]

            yield v

            pcba = sorted(v.possible_backward_connections())
            for f, t in pcba:
                d = v.copy()
                d.action(f'cb {f} {t}', reset=False)
                # print(f'Checking new action stack: {d.action_stack}')
                if d not in visited:
                    visited.append(d)
                    queue.append(d)
            if v.is_in_vertex_mode:
                d = v.copy()
                d.action('nv', reset=False)
                # print(f'Checking new action stack: {d.action_stack}')
                if d not in visited:
                    visited.append(d)
                    queue.append(d)
    def get_denominators(self) -> list[int]:
        ev = self.graph.evaluate()
        output_flow = ev['output_flow']
        den_list = []
        for _, flow in output_flow.items():
            for _, frac in flow.items():
                den_list.append(frac.denominator)
        return list(sorted(set(den_list)))
    def __hash__(self):
        return hash(self.graph)
    def __eq__(self, other):
        if not isinstance(other, GraphDesigner):
            return False
        return hash(other) == hash(self)
    def __repr__(self):
        return f'{repr(self.graph.evaluate())}\nOpen outputs: {list(sorted(set(self.open_outputs())))}\n(open outputs depth regression): {list(sorted(set(map(self.get_cb_amount, self.open_outputs()))))}\nOpen inputs: {list(sorted(set(self.open_inputs())))}\n(open inputs depth regression): {list(sorted(set(map(self.get_cb_amount, self.open_inputs()))))}\nPossible cb pairs: {list(sorted(self.possible_backward_connections()))}'


In [19]:
des = GraphDesigner()
print(des)

Accuracy: 1.00000
Error: 0.00000
Score: 1.00000
Bottlenecks: []
Output flow:
2:	0:1/2
3:	0:1/2

Open outputs: []
(open outputs depth regression): []
Open inputs: [1]
(open inputs depth regression): [0]
Possible cb pairs: []


In [20]:
des.action('nv -> nv -> cb 0 1 -> cb 1 2')
print(des)

Accuracy: 0.00000
Error: 0.60801
Score: -0.06080
Bottlenecks: []
Output flow:
2:	0:3/4
7:	0:1/4

Open outputs: [5]
(open outputs depth regression): [0]
Open inputs: [5]
(open inputs depth regression): [0]
Possible cb pairs: []


In [21]:
des.action('nv -> nv -> nv -> cb 0 1 -> cb 0 3')
print(des)

Accuracy: 0.00000
Error: 4.54876
Score: -0.45488
Bottlenecks: []
Output flow:
2:	0:6/11
4:	0:3/11
6:	0:2/11

Open outputs: [3, 5]
(open outputs depth regression): [1, 2]
Open inputs: [3, 7]
(open inputs depth regression): [0, 2]
Possible cb pairs: [(1, 2)]


In [29]:
odds = {}

for d in des.bfs():
    dens = d.get_denominators()
    for den in dens:
        if den % 2 == 1:
            if den not in odds:
                if isprime(den):
                    print()
                    print(d.action_stack)
                    print(den)
                odds[den] = [d.action_stack]
                with open('./action_stacks.txt', 'w') as f:
                    f.write(
                        '\n\n'.join(
                            f'1/{_denom}:\n' + '\n'.join(action_stacks)
                            for _denom, action_stacks in sorted(odds.items())
                        )
                    )
            else:
                if len(odds[den]) < 8:
                    odds[den].append(d.action_stack)



nv -> cb 0 1
3

nv -> nv -> cb 0 2
7

nv -> nv -> cb 0 1 -> cb 0 2
5

nv -> nv -> nv -> cb 0 1 -> cb 0 3
11

nv -> nv -> nv -> cb 0 2 -> cb 0 3
13

nv -> nv -> nv -> nv -> cb 0 4
31

nv -> nv -> nv -> nv -> cb 0 1 -> cb 0 4
23

nv -> nv -> nv -> nv -> cb 0 3 -> cb 0 4
29

nv -> nv -> nv -> nv -> cb 0 1 -> cb 0 2 -> cb 2 4
17

nv -> nv -> nv -> nv -> cb 0 1 -> cb 0 3 -> cb 2 4
19

nv -> nv -> nv -> nv -> nv -> cb 0 1 -> cb 0 5
47

nv -> nv -> nv -> nv -> nv -> cb 0 3 -> cb 0 5
59

nv -> nv -> nv -> nv -> nv -> cb 0 4 -> cb 0 5
61

nv -> nv -> nv -> nv -> nv -> nv -> cb 0 6
127

nv -> nv -> nv -> nv -> nv -> cb 0 1 -> cb 0 2 -> cb 2 5
37

nv -> nv -> nv -> nv -> nv -> cb 0 1 -> cb 0 3 -> cb 2 5
41

nv -> nv -> nv -> nv -> nv -> cb 0 1 -> cb 0 4 -> cb 2 5
43

nv -> nv -> nv -> nv -> nv -> cb 0 4 -> cb 0 5 -> cb 1 3
53

nv -> nv -> nv -> nv -> nv -> nv -> cb 0 1 -> cb 0 4 -> cb 2 6
89

nv -> nv -> nv -> nv -> nv -> nv -> cb 0 1 -> cb 0 6 -> cb 1 3
79

nv -> nv -> nv -> nv -> nv -> nv -> c

KeyboardInterrupt: 

In [14]:
_1ns = {}
with open('graph_des.txt', 'w') as out_f:
    with open('action_stacks.txt', 'r') as f:
        nl = None
        for line in f:
            line = line.strip()
            if line == '':
                continue
            if nl is not None:
                des = GraphDesigner()
                des.action(line)
                print(des, file=out_f)
                _1ns[nl] = des.graph.compact_string
                nl = None
            if line[-1] == ':':
                n = int(line.split('/')[1][:-1])
                nl = n

In [18]:
json.dump(_1ns, open('1ns.json', 'w'), indent=4)

In [21]:
des = GraphDesigner()
des.action('nv -> nv -> nv -> nv -> nv -> nv -> cb 0 4 -> cb 0 5 -> cb 3 6')
print(des)
print(des.graph.compact_string)

Accuracy: 0.00000
Error: 7.75725
Score: -0.77572
Bottlenecks: []
Output flow:
2:	0:61/114
4:	0:31/114
6:	0:8/57
10:	0:2/57
12:	0:1/57

Open outputs: [3, 5, 9, 11]
(open outputs depth regression): [1, 2, 4, 5]
Open inputs: [7, 9, 11, 13]
(open inputs depth regression): [0, 1, 2, 3]
Possible cb pairs: [(1, 2), (1, 3), (2, 3)]
eJwVysERACAIA7BVOkAfFETPuv9e6juJkxycVFB5woKcLJQHG+3JheVNYfuf57KSKqjcrAuaIQxi
